In [1]:
#!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz
#!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [2]:
import os, pathlib, shutil, random

if False:
    base_dir = pathlib.Path('aclImdb')
    val_dir = base_dir / 'val'
    train_dir = base_dir / 'train'
    for category in ('neg', 'pos'):
        os.makedirs(val_dir / category)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)
        n_val_samples = int(0.2 * len(files))
        val_files = files[-n_val_samples:]
        for filename in val_files:
            shutil.move(train_dir / category / filename, val_dir / category / filename)

In [3]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [4]:
for inputs, targets in train_ds:
    print(inputs.shape, inputs.dtype, inputs[0])
    print(targets.shape, targets.dtype, targets[0])
    break

(32,) <dtype: 'string'> tf.Tensor(b'So much is wrong with this abysmal little wet fart of a movie that it\'s hard to know where to begin.<br /><br />First of all, it\'s a remarkably un-scary scary movie, even by Amercian standards. The dialogue is clich\xc3\xa9, the characters are two-dimensional, the writing is ho-hum, and what little story there is is neither coherent nor remotely interesting.<br /><br />We meet the following stereotypes in order: Balding Loser Guy (probably divorced, but who knows? This movie doesn\'t tell us) with a brave heart, the Young Hero (who doesn\'t do anything heroic at all), Brave Little Kid (with a homicidal streak a mile wide) and Black Bad-Ass Bitch (with more brawn than brains). These guys take up an ongoing fight with the Tall Scary Reaper Man and his evil Ewoks.<br /><br />Oh, and the film is full of wicked little metal orbs whoosing around menacing people. Given a chance, they perform impromptu brain surgery on those who doen\'t have the mental acu

In [5]:
from keras import layers

text_vectorization = layers.TextVectorization(max_tokens=20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

In [6]:
for inputs, targets in binary_1gram_train_ds:
    print(inputs.shape, inputs.dtype, inputs[0])
    print(targets.shape, targets.dtype, targets[0])
    break

(32, 20000) <dtype: 'float32'> tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
(32,) <dtype: 'int32'> tf.Tensor(1, shape=(), dtype=int32)


In [7]:
import keras
from keras import layers, activations, optimizers, losses, metrics, callbacks

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = layers.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation=activations.relu)(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation=activations.sigmoid)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
    return model

In [8]:
model = get_model()
model.summary()
callback_list = [
    callbacks.ModelCheckpoint('binary_1gram', save_best_only=True)
]

model.fit(binary_1gram_train_ds, epochs=10, validation_data=binary_1gram_val_ds, callbacks=callback_list)

model = keras.models.load_model('binary_1gram')
print(model.evaluate(binary_1gram_test_ds))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
605/625 [============================>.] - ETA: 0s - loss: 0.4079 - binary_accuracy: 0.8250INFO:tensorflow:Assets written to: binary_1gram/assets


INFO:tensorflow:Assets written to: binary_1gram/assets


625/625 [==============================] - 2s 2ms/step - loss: 0.4054 - binary_accuracy: 0.8264 - val_loss: 0.2701 - val_binary_accuracy: 0.8956
Epoch 2/10
598/625 [===========================>..] - ETA: 0s - loss: 0.2763 - binary_accuracy: 0.8965INFO:tensorflow:Assets written to: binary_1gram/assets


INFO:tensorflow:Assets written to: binary_1gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.2757 - binary_accuracy: 0.8970 - val_loss: 0.2637 - val_binary_accuracy: 0.9002
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2431 - binary_accuracy: 0.9150 - val_loss: 0.2766 - val_binary_accuracy: 0.9006
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2320 - binary_accuracy: 0.9218 - val_loss: 0.2890 - val_binary_accuracy: 0.8972
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2252 - binary_accuracy: 0.9254 - val_loss: 0.3003 - val_binary_accuracy: 0.8984
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2172 - binary_accuracy: 0.9271 - val_loss: 0.3084 - val_binary_accuracy: 0.8942
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2213 - binary_accuracy: 0.9309 - val_loss: 0.3193 - val_binary_accuracy: 0.8924
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0

In [9]:
text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='multi_hot')
text_vectorization.adapt(text_only_train_ds)

In [10]:
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

model = get_model()
callback_list = [
    callbacks.ModelCheckpoint('binary_2gram', save_best_only=True)
]

model.fit(binary_2gram_train_ds, validation_data=binary_2gram_val_ds, epochs=10, callbacks=callback_list)

model = keras.models.load_model('binary_2gram')
print(model.evaluate(binary_2gram_test_ds))

Epoch 1/10
617/625 [============================>.] - ETA: 0s - loss: 0.3904 - binary_accuracy: 0.8384INFO:tensorflow:Assets written to: binary_2gram/assets


INFO:tensorflow:Assets written to: binary_2gram/assets


625/625 [==============================] - 2s 2ms/step - loss: 0.3889 - binary_accuracy: 0.8393 - val_loss: 0.2497 - val_binary_accuracy: 0.9040
Epoch 2/10
608/625 [============================>.] - ETA: 0s - loss: 0.2464 - binary_accuracy: 0.9136INFO:tensorflow:Assets written to: binary_2gram/assets


INFO:tensorflow:Assets written to: binary_2gram/assets


625/625 [==============================] - 2s 3ms/step - loss: 0.2469 - binary_accuracy: 0.9136 - val_loss: 0.2478 - val_binary_accuracy: 0.9092
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2103 - binary_accuracy: 0.9321 - val_loss: 0.2554 - val_binary_accuracy: 0.9090
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1928 - binary_accuracy: 0.9362 - val_loss: 0.2784 - val_binary_accuracy: 0.9076
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1921 - binary_accuracy: 0.9433 - val_loss: 0.2818 - val_binary_accuracy: 0.9092
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1777 - binary_accuracy: 0.9442 - val_loss: 0.2969 - val_binary_accuracy: 0.9054
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.1856 - binary_accuracy: 0.9474 - val_loss: 0.3031 - val_binary_accuracy: 0.9062
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0

In [11]:
text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='tf_idf')
text_vectorization.adapt(text_only_train_ds)

In [12]:
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

callback_list = [
    callbacks.ModelCheckpoint('tfidf_2gram', save_best_only=True)
]

model = get_model()
model.fit(tfidf_2gram_train_ds.cache(), validation_data=tfidf_2gram_val_ds, epochs=10, callbacks=callback_list)

model = keras.models.load_model('tfidf_2gram')
print(model.evaluate(tfidf_2gram_test_ds))

Epoch 1/10
610/625 [============================>.] - ETA: 0s - loss: 0.5574 - binary_accuracy: 0.7677INFO:tensorflow:Assets written to: tfidf_2gram/assets


INFO:tensorflow:Assets written to: tfidf_2gram/assets


625/625 [==============================] - 2s 2ms/step - loss: 0.5539 - binary_accuracy: 0.7700 - val_loss: 0.3117 - val_binary_accuracy: 0.8922
Epoch 2/10
602/625 [===========================>..] - ETA: 0s - loss: 0.3586 - binary_accuracy: 0.8596INFO:tensorflow:Assets written to: tfidf_2gram/assets


INFO:tensorflow:Assets written to: tfidf_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.3582 - binary_accuracy: 0.8598 - val_loss: 0.2766 - val_binary_accuracy: 0.8900
Epoch 3/10
625/625 [==============================] - 1s 2ms/step - loss: 0.3220 - binary_accuracy: 0.8719 - val_loss: 0.2956 - val_binary_accuracy: 0.8724
Epoch 4/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2977 - binary_accuracy: 0.8839 - val_loss: 0.2967 - val_binary_accuracy: 0.8746
Epoch 5/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2876 - binary_accuracy: 0.8892 - val_loss: 0.3007 - val_binary_accuracy: 0.8906
Epoch 6/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2629 - binary_accuracy: 0.8973 - val_loss: 0.3053 - val_binary_accuracy: 0.8948
Epoch 7/10
625/625 [==============================] - 1s 2ms/step - loss: 0.2555 - binary_accuracy: 0.9041 - val_loss: 0.3097 - val_binary_accuracy: 0.8928
Epoch 8/10
625/625 [==============================] - 1s 2ms/step - loss: 0

In [13]:
max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

In [14]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype=tf.int64)
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

callback_list = [
    callbacks.ModelCheckpoint('one_hot_bidir_lstm', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callback_list)

model = keras.models.load_model('one_hot_bidir_lstm')
print(model.evaluate(int_test_ds))

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

INFO:tensorflow:Assets written to: one_hot_bidir_lstm/assets


782/782 [==============================] - 594s 760ms/step - loss: 0.3816 - binary_accuracy: 0.8532
[0.38155484199523926, 0.8532000184059143]


In [15]:
inputs = keras.Input(shape=(None,), dtype=tf.int64)
x = tf.one_hot(inputs, depth=max_tokens)
x = layers.SeparableConv1D(filters=64, kernel_size=3, activation=activations.relu)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.SeparableConv1D(filters=64, kernel_size=3, activation=activations.relu)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

callback_list = [
    callbacks.ModelCheckpoint('one_hot_conv', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callback_list)

model = keras.models.load_model('one_hot_conv')
print(model.evaluate(int_test_ds))

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_1 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 separable_conv1d (Separabl  (None, None, 64)          1340064   
 eConv1D)                                                        
                                                                 
 max_pooling1d (MaxPooling1  (None, None, 64)          0         
 D)                                                              
                                                                 
 separable_conv1d_1 (Separa  (None, None, 64)          4352      
 bleConv1D)                                                      
                                                           

INFO:tensorflow:Assets written to: one_hot_conv/assets


782/782 [==============================] - 191s 244ms/step - loss: 0.6931 - binary_accuracy: 0.5000
[0.693149209022522, 0.5]


In [16]:
inputs = layers.Input(shape=(None,), dtype=tf.int64)
embbedded = layers.Embedding(input_dim=max_tokens, output_dim=128)(inputs)
x = layers.Bidirectional(layers.LSTM(units=32))(embbedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])

callback_list = [
    callbacks.ModelCheckpoint('embeddings_bidir_lstm', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callback_list)
model = keras.models.load_model('embeddings_bidir_lstm')
print(model.evaluate(int_test_ds))

Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 0.5048 - binary_accuracy: 0.7656INFO:tensorflow:Assets written to: embeddings_bidir_lstm/assets


INFO:tensorflow:Assets written to: embeddings_bidir_lstm/assets


625/625 [==============================] - 84s 133ms/step - loss: 0.5048 - binary_accuracy: 0.7656 - val_loss: 0.3292 - val_binary_accuracy: 0.8718
Epoch 2/10
625/625 [==============================] - ETA: 0s - loss: 0.3267 - binary_accuracy: 0.8838INFO:tensorflow:Assets written to: embeddings_bidir_lstm/assets


INFO:tensorflow:Assets written to: embeddings_bidir_lstm/assets


625/625 [==============================] - 83s 133ms/step - loss: 0.3267 - binary_accuracy: 0.8838 - val_loss: 0.3270 - val_binary_accuracy: 0.8826
Epoch 3/10
625/625 [==============================] - 81s 129ms/step - loss: 0.2493 - binary_accuracy: 0.9111 - val_loss: 0.3371 - val_binary_accuracy: 0.8850
Epoch 4/10
625/625 [==============================] - 83s 133ms/step - loss: 0.2184 - binary_accuracy: 0.9255 - val_loss: 0.3790 - val_binary_accuracy: 0.8476
Epoch 5/10
625/625 [==============================] - ETA: 0s - loss: 0.1805 - binary_accuracy: 0.9391INFO:tensorflow:Assets written to: embeddings_bidir_lstm/assets


INFO:tensorflow:Assets written to: embeddings_bidir_lstm/assets


625/625 [==============================] - 89s 143ms/step - loss: 0.1805 - binary_accuracy: 0.9391 - val_loss: 0.3008 - val_binary_accuracy: 0.8866
Epoch 6/10
625/625 [==============================] - 81s 129ms/step - loss: 0.1587 - binary_accuracy: 0.9496 - val_loss: 0.3678 - val_binary_accuracy: 0.8722
Epoch 7/10
625/625 [==============================] - 79s 127ms/step - loss: 0.1373 - binary_accuracy: 0.9552 - val_loss: 0.3325 - val_binary_accuracy: 0.8904
Epoch 8/10
625/625 [==============================] - 82s 131ms/step - loss: 0.1221 - binary_accuracy: 0.9597 - val_loss: 0.3690 - val_binary_accuracy: 0.8836
Epoch 9/10
625/625 [==============================] - 80s 128ms/step - loss: 0.1040 - binary_accuracy: 0.9663 - val_loss: 0.3578 - val_binary_accuracy: 0.8766
Epoch 10/10
782/782 [==============================] - 20s 26ms/step - loss: 0.3998 - binary_accuracy: 0.8520
[0.39976829290390015, 0.8519600033760071]


In [17]:
inputs = layers.Input(shape=(None,), dtype=tf.int64)
embbedded = layers.Embedding(input_dim=max_tokens, output_dim=128, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(units=32))(embbedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])

callback_list = [
    callbacks.ModelCheckpoint('embeddings_bidir_lstm_masking', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callback_list)
model = keras.models.load_model('embeddings_bidir_lstm_masking')
print(model.evaluate(int_test_ds))

Epoch 1/10
625/625 [==============================] - ETA: 0s - loss: 0.4053 - binary_accuracy: 0.8127INFO:tensorflow:Assets written to: embeddings_bidir_lstm_masking/assets


INFO:tensorflow:Assets written to: embeddings_bidir_lstm_masking/assets


625/625 [==============================] - 110s 172ms/step - loss: 0.4053 - binary_accuracy: 0.8127 - val_loss: 0.2826 - val_binary_accuracy: 0.8910
Epoch 2/10
625/625 [==============================] - ETA: 0s - loss: 0.2450 - binary_accuracy: 0.9064INFO:tensorflow:Assets written to: embeddings_bidir_lstm_masking/assets


INFO:tensorflow:Assets written to: embeddings_bidir_lstm_masking/assets


625/625 [==============================] - 102s 164ms/step - loss: 0.2450 - binary_accuracy: 0.9064 - val_loss: 0.2809 - val_binary_accuracy: 0.8920
Epoch 3/10
625/625 [==============================] - 98s 156ms/step - loss: 0.1812 - binary_accuracy: 0.9331 - val_loss: 0.3054 - val_binary_accuracy: 0.8926
Epoch 4/10
625/625 [==============================] - 98s 156ms/step - loss: 0.1404 - binary_accuracy: 0.9486 - val_loss: 0.3428 - val_binary_accuracy: 0.8910
Epoch 5/10
625/625 [==============================] - 98s 157ms/step - loss: 0.1104 - binary_accuracy: 0.9618 - val_loss: 0.3024 - val_binary_accuracy: 0.8836
Epoch 6/10
625/625 [==============================] - 100s 160ms/step - loss: 0.0852 - binary_accuracy: 0.9703 - val_loss: 0.3545 - val_binary_accuracy: 0.8920
Epoch 7/10
625/625 [==============================] - 96s 153ms/step - loss: 0.0622 - binary_accuracy: 0.9789 - val_loss: 0.3745 - val_binary_accuracy: 0.8918
Epoch 8/10
625/625 [==============================] - 9

2023-09-21 14:52:56.106021: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 14:52:56.181669: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 14:52:56.628445: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 14:52:56.633595: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 14:52:56.671454: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _ou

782/782 [==============================] - 24s 30ms/step - loss: 0.3484 - binary_accuracy: 0.8704
[0.3483661413192749, 0.8703600168228149]


In [21]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

In [23]:
import numpy as np
path_to_glove_file = 'glove.6B.100d.txt'

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print(len(embeddings_index))

400000


In [33]:
(embeddings_index['queen'] - embeddings_index['king']) - (embeddings_index['princess'] - embeddings_index['prince'])

array([-0.17083003,  0.59597003,  0.59098995,  0.02536997,  0.61793995,
       -0.19281995, -0.089395  ,  0.63546395, -0.21893004,  0.87589   ,
        0.138769  , -0.25664997, -0.27886   ,  0.19888899,  0.482831  ,
        0.00455004, -0.53747606,  0.03740007, -0.17844999, -0.8905601 ,
       -0.53164005,  0.51741993,  0.50496995, -1.15989   , -0.35557997,
       -0.32298994, -0.39193   , -0.02756   ,  0.81107   ,  0.00885099,
       -0.10747302, -0.04339998,  0.23818994,  0.06065002, -0.07598999,
        0.09676903, -0.34658   ,  0.28506997, -0.982826  , -0.00301   ,
        0.41085   , -0.17233601, -0.01127499, -0.16784   ,  0.99841   ,
        0.09046301, -0.03593001, -0.24147004, -0.21476996,  0.13638696,
        0.1085    , -0.41917497, -0.54655004,  0.06991002, -0.47669   ,
       -0.28980005, -0.09448004, -0.71713   , -0.19064   , -0.181496  ,
       -0.36639   , -0.48315996, -0.01811402,  0.109236  ,  0.398004  ,
       -0.08116996,  0.21427998,  0.09854996,  0.553653  ,  0.13

In [34]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [38]:
embedding_layer = layers.Embedding(
    input_dim=max_tokens,
    output_dim=embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True
)

In [39]:
inputs = layers.Input(shape=(None,), dtype=tf.int64)
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

callback_list = [
    callbacks.ModelCheckpoint('glove_embeddings_lstm', save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callback_list)


model = keras.models.load_model('glove_embeddings_lstm')
print(model.evaluate(int_test_ds))

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 1)                 65        
                                                                 
Total params: 2034113 (7.76 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 2000000 (7.63 MB)
_________________

INFO:tensorflow:Assets written to: glove_embeddings_lstm/assets


625/625 [==============================] - 90s 140ms/step - loss: 0.5703 - binary_accuracy: 0.6988 - val_loss: 0.4237 - val_binary_accuracy: 0.8090
Epoch 2/10
625/625 [==============================] - 76s 121ms/step - loss: 0.4463 - binary_accuracy: 0.7965 - val_loss: 0.4770 - val_binary_accuracy: 0.7664
Epoch 3/10
625/625 [==============================] - ETA: 0s - loss: 0.4019 - binary_accuracy: 0.8241INFO:tensorflow:Assets written to: glove_embeddings_lstm/assets


INFO:tensorflow:Assets written to: glove_embeddings_lstm/assets


625/625 [==============================] - 85s 137ms/step - loss: 0.4019 - binary_accuracy: 0.8241 - val_loss: 0.3656 - val_binary_accuracy: 0.8360
Epoch 4/10
625/625 [==============================] - ETA: 0s - loss: 0.3674 - binary_accuracy: 0.8433INFO:tensorflow:Assets written to: glove_embeddings_lstm/assets


...

625/625 [==============================] - 84s 135ms/step - loss: 0.3055 - binary_accuracy: 0.8736 - val_loss: 0.2868 - val_binary_accuracy: 0.8818
Epoch 8/10
625/625 [==============================] - 74s 118ms/step - loss: 0.2906 - binary_accuracy: 0.8824 - val_loss: 0.2925 - val_binary_accuracy: 0.8806
Epoch 9/10
625/625 [==============================] - 73s 117ms/step - loss: 0.2791 - binary_accuracy: 0.8880 - val_loss: 0.3084 - val_binary_accuracy: 0.8704
Epoch 10/10
625/625 [==============================] - ETA: 0s - loss: 0.2611 - binary_accuracy: 0.8953INFO:tensorflow:Assets written to: glove_embeddings_lstm/assets


INFO:tensorflow:Assets written to: glove_embeddings_lstm/assets


625/625 [==============================] - 83s 134ms/step - loss: 0.2611 - binary_accuracy: 0.8953 - val_loss: 0.2782 - val_binary_accuracy: 0.8864


2023-09-21 15:32:34.412863: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 15:32:34.485160: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 15:32:34.922968: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 15:32:35.560045: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2023-09-21 15:32:35.840708: W tensorflow/core/common_runtime/graph_constructor.cc:834] Node 'cond/while' has 13 outputs but the _output_sh

782/782 [==============================] - 23s 28ms/step - loss: 0.2957 - binary_accuracy: 0.8764
[0.29573968052864075, 0.8763999938964844]
